In [ ]:
import os
import sys
import math
import webbrowser
from pathlib import Path
from abc import abstractmethod
from typing import Literal, List, Any

from IPython.display import clear_output

import sqlite3
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import qualitative, sequential

lmt_analysis_path = Path.cwd().parent
sys.path.append(lmt_analysis_path.as_posix())

from dim_c_brains.scripts.reports_manager import HTMLReportManager
from dim_c_brains.scripts.events import generic_events_list

from lmtanalysis.Animal import Animal, AnimalPool
from lmtanalysis.Measure import oneDay, oneHour, oneMinute
from lmtanalysis.Event import EventTimeLine
from lmtanalysis.ParametersMouse import ParametersMouse

COLORS = {
    "discrete": qualitative.Bold[::-1],
    "continuous": sequential.Plotly3
}

# data_path = Path.cwd() / "res" / "data" / "exemple data - 20180110_validation_4_ind_Experiment_6644_e.sqlite"
data_path = Path.home() / "Syncnot" / "lmt-blocks" / "experiments" / "xmd" / "nadege" / "groupe1-cage1-LMT1.sqlite"

In [11]:
# ref_time is your reference pd.Timestamp (for frame 0)
ref_time = pd.Timestamp('2025-12-17 10:03:27')

# For 1-minute bins, get the previous bin edge:
first_bin = ref_time.floor('1440min')

# If you want the bin edge strictly before ref_time (not including it if exact):
if ref_time == first_bin:
    first_bin = first_bin - pd.Timedelta(minutes=1)

print(first_bin)

2025-12-17 00:00:00


In [ ]:
connection = sqlite3.connect(data_path.as_posix())

# reports_manager = HTMLReportManager(exp_name="example_analysis")
start_time = 12*oneHour
end_time = 13*oneHour


animal_pool = AnimalPool()
animal_pool.loadAnimals(connection)
animal_pool.loadDetection(start= start_time, end= end_time)
animal_pool.conn
for animal in animal_pool.animalDictionary.values():
    speeds = {}
    detection_frames = sorted(animal.detectionDictionary.keys())
    for f in detection_frames[1:]:
        previous_pos = animal.detectionDictionary.get(f-1)
        current_pos = animal.detectionDictionary.get(f)

        if (current_pos is None or previous_pos is None):
            continue
        
        # cm/s
        speeds[f] = math.hypot(
            current_pos.massX - previous_pos.massX,
            current_pos.massY - previous_pos.massY
            )*animal.parameters.scaleFactor*30
    animal.speeds = speeds
print("Loading Done\n")

clear_output()

print(f"Speeds memory usage: {sys.getsizeof(animal_pool.animalDictionary[1].speeds)/10**6:.2f} Mo")

Speeds memory usage: 0.59 Mo


# Distance vs Time

In [4]:
bins = list(range(start_time, animal.getMaxDetectionT()//2, 5*oneMinute))

dist = []

for animal in animal_pool.animalDictionary.values():
    dist.append(animal.getDistancePerBin(oneMinute, start_time, end_time, filter_flickering= True, filter_stop= True))

clear_output()

In [ ]:
animal.time

clear_output()

In [ ]:
df_all.head()

In [ ]:
fig = px.bar_polar(
    df_all, r="count", theta="event",
    color="animal_id", color_discrete_sequence= COLORS["discrete"]
)

fig.update_layout(
    polar=dict(
        radialaxis=dict(title=dict(text="count"))
    ),
    legend_title="Animal ID"
)

In [ ]:
fig = px.histogram(
    df_all, x="event", y="count", barmode= "group",
    color="animal_id", color_discrete_sequence= COLORS["discrete"]
)

fig.update_layout(
    polar=dict(
        radialaxis=dict(title=dict(text="count"))
    ),
    legend_title="Animal ID"
)

In [ ]:
query = """
SELECT FRAME.TIMESTAMP, ANIMAL.RFID, DETECTION.MASS_X, DETECTION.MASS_Y
FROM DETECTION
JOIN FRAME ON DETECTION.FRAMENUMBER = FRAME.FRAMENUMBER
JOIN ANIMAL ON DETECTION.ANIMALID = ANIMAL.ID
WHERE DETECTION.FRAMENUMBER >= 0 AND DETECTION.FRAMENUMBER <= 500000
"""
df = pd.read_sql_query(query, connection)
df["POS"] = (df["MASS_X"]**2 + df["MASS_Y"]**2).pow(0.5)

In [ ]:
df["DATETIME"] = pd.to_datetime(df["TIMESTAMP"], unit='ms')
df.head()

In [ ]:
px.line(df, x="DATETIME", y="POS", color="RFID")

In [ ]:
query = """
SELECT 
    (FRAME.TIMESTAMP / 1000 / 60 / 15) AS interval_15min,
    ANIMAL.RFID,
    AVG(DETECTION.MASS_X) AS mean_mass_x,
    AVG(DETECTION.MASS_Y) AS mean_mass_y,
    AVG((DETECTION.MASS_X * DETECTION.MASS_X + DETECTION.MASS_Y * DETECTION.MASS_Y)) AS mean_pos_sq
FROM DETECTION
JOIN FRAME ON DETECTION.FRAMENUMBER = FRAME.FRAMENUMBER
JOIN ANIMAL ON DETECTION.ANIMALID = ANIMAL.ID
WHERE DETECTION.FRAMENUMBER >= 0 AND DETECTION.FRAMENUMBER <= 500000
GROUP BY interval_15min, ANIMAL.RFID
ORDER BY interval_15min
"""
df = pd.read_sql_query(query, connection)
df["interval_start"] = pd.to_datetime(df["interval_15min"] * 15 * 60, unit="s")
df["mean_pos"] = df["mean_pos_sq"].pow(0.5)

In [ ]:
px.line(df, x="interval_start", y="mean_pos", color="RFID")

In [ ]:
query = """
SELECT
    interval_15min,
    RFID,
    SUM(displacement) AS sum_displacement
FROM (
    SELECT
        (FRAME.TIMESTAMP / 1000 / 60 / 15) AS interval_15min,
        ANIMAL.RFID,
        sqrt(
            (DETECTION.MASS_X - LAG(DETECTION.MASS_X) OVER (PARTITION BY DETECTION.ANIMALID ORDER BY DETECTION.FRAMENUMBER)) *
            (DETECTION.MASS_X - LAG(DETECTION.MASS_X) OVER (PARTITION BY DETECTION.ANIMALID ORDER BY DETECTION.FRAMENUMBER)) +
            (DETECTION.MASS_Y - LAG(DETECTION.MASS_Y) OVER (PARTITION BY DETECTION.ANIMALID ORDER BY DETECTION.FRAMENUMBER)) *
            (DETECTION.MASS_Y - LAG(DETECTION.MASS_Y) OVER (PARTITION BY DETECTION.ANIMALID ORDER BY DETECTION.FRAMENUMBER))
        ) AS displacement
    FROM DETECTION
    JOIN FRAME ON DETECTION.FRAMENUMBER = FRAME.FRAMENUMBER
    JOIN ANIMAL ON DETECTION.ANIMALID = ANIMAL.ID
    WHERE DETECTION.FRAMENUMBER >= 0 AND DETECTION.FRAMENUMBER <= 500000
)
WHERE displacement IS NOT NULL
GROUP BY interval_15min, RFID
ORDER BY interval_15min
"""
df = pd.read_sql_query(query, connection)
df.head(7)

In [ ]:
df["time"] = pd.to_datetime(df["interval_15min"] * 15 * 60, unit="s")
px.line(df, x="time", y="sum_displacement", color="RFID")